In [7]:
import numpy as np
import itertools

In [35]:
# HÀM NÀY CHUYỂN BIỂU THỨC TỪ TRUNG TỐ SANG HÂU TỐ
def InFix2PostFix(s):
    def reject(what): # Produce a readable error
        raise SyntaxError("Expected {}, but got {} at index {}".format(
            what or "EOF", 
            "'{}'".format(tokens[-1]) if tokens else "EOF", 
            len(s) - len(tokens)
        ))

    get = lambda: tokens.pop() if tokens else ""
    put = lambda token: output.append(token)
    match = lambda what: tokens[-1] in what if tokens else what == ""
    expect = lambda what: get() if match(what) else reject(what)

    def suffix():
        token = get()
        term()
        put(token)

    def parens(): 
        expect("(")
        expression(")")

    def term():
        if match(identifier): put(get())
        elif match(unary): suffix()
        elif match("("): parens()
        else: expect("an identifier, a unary operator or an opening parenthesis");

    def expression(terminator):
        term()
        if match(binary): suffix()
        expect(terminator)

    # Define the token groups
    identifier = "abcdefghijklmnopqrstuwxyz"
    identifier += identifier.upper()
    unary = "~";
    binary = "&|→>=";
    tokens = list(reversed(s)) # More efficient to pop from the end
    output = [] # Will be populated during the parsing
    expression("") # Parse!
    return "".join(output)

In [36]:
# HÀM NÀY DÙNG ĐỂ SINH RA CÁC BIỂU THỨC CON ĐÃ TỔNG HỢP NÊN BIỂU THỨC CHÍNH
def create_sub_expression(postfix):
    final_postfix = []
    for i in postfix:
        #print(i)
        final_postfix.append(i)
    stack = []
    result = []
    #print(final_postfix)
    while(len(final_postfix) != 0):
        # lấy từ queue postfix 
        character = final_postfix[0]
        #print('çharacter: ', character)
        del final_postfix[0]
        
        # cho vào stack
        if character in ['~', '&', '|', '>', '=',]:
            if character != '~': # tao bieu thuc su ly hai toan han
                toanhang2 = stack[-1]
                del stack[-1]
                toanhang1 = stack[-1]
                del stack[-1]
                #print('toanhang1: ',toanhang1, ' toanhang2: ', toanhang2, ' toantu:', character)
                expression = toanhang1 + character + toanhang2
                result.append(expression)
                expression = '(' + expression + ')'
                stack.append(expression)
            elif character == '~': # tao bieu thuc su ly mot toan han
                toanhang = stack[-1]
                del stack[-1]
                #print('toanhang: ',toanhang, ' toantu:', character)
                expression = character + toanhang
                result.append(expression)
                expression = expression
                stack.append(expression)
        else:
            stack.append(character)
    #print('stack: ', stack)
    #print('result: ', result)
    return stack, result

In [37]:
# HÀM NÀY TẠO HEADER CHO BẢNG CHÂN TRỊ
def createHeader(expression):
    operand = []
    for i in expression:
        if(i not in ['~', '&', '|', '>', '=', '(', ')']):
            operand.append(i)
    operand = list(set(operand))
    operand.sort()
    stack, result = create_sub_expression(expression)
    header = operand + result
    return header

In [38]:
def tinh_toan_hai_toan_hang(dic_input, toanhang1, toanhang2, toantu):
    if(toanhang1 != False and toanhang1 != True):
        toanhang1 = dic_input[toanhang1]
    if(toanhang2 != False and toanhang2 != True):
        toanhang2 = dic_input[toanhang2]
    if(toantu == '&'):
        return toanhang1 and toanhang2
    elif(toantu == '|'):
        return toanhang1 or toanhang2
    elif(toantu == '>'):
        if toanhang1:
            return toanhang2
        else :
            return True
    elif(toantu == '='):
        return toanhang1 == toanhang2

In [39]:
def tinh_toan_mot_toan_hang(dic_input, toanhang, toantu):
    if(toanhang != False and toanhang != True):
        toanhang = dic_input[toanhang]
    if(toantu == '~'):
        return not toanhang

In [40]:
# HÀM NÀY TẠO BODY CHO BẢNG CHÂN TRỊ
def createBody(postfix):
    #bieuthuc = sinh_bieu_thuc(postfix)[0]
    final_postfix = []
    backup_final_postfix = []
    toanhang = []
    for i in postfix:
        final_postfix.append(i)
        backup_final_postfix.append(i)
        if(i not in ['~', '&', '|', '>', '=', '(', ')']):
            toanhang.append(i)
    toanhang = list(set(toanhang))
    toanhang.sort()
    #print('toan hang: ', toanhang)
    stack = []
    result = []
    
    input = list(itertools.product([False,True],repeat=len(toanhang)))
    dic_input = {}
    list_dic_input = []
    for i in input:
        for j in range(len(toanhang)):      
            dic_input[toanhang[j]] = i[j]
        list_dic_input.append(dic_input)
        dic_input = {}
    result_item = []
    for dic_input in list_dic_input:
        for j in dic_input:
            result_item.append(dic_input[j])
        #print(dic_input)
        while(len(final_postfix) != 0):
            # lấy từ queue postfix 
            character = final_postfix[0]
            del final_postfix[0]

            # cho vào stack
            if character in ['~', '&', '|', '>', '=',]:
                if character != '~': # tao bieu thuc su ly hai toan han
                    #print('tinh toan 2')
                    toanhang2 = stack[-1]
                    del stack[-1]
                    toanhang1 = stack[-1]
                    del stack[-1]
                    #print('toanhang1: ',toanhang1, ' toanhang2: ', toanhang2, ' toantu:', character)
                    re = tinh_toan_hai_toan_hang(dic_input, toanhang1, toanhang2, character)
                    #print('ket qua:' , re)
                    result_item.append(re)
                    stack.append(re)
                elif character == '~': # tao bieu thuc su ly mot toan han
                    #print('tinh toan 1')
                    toanhang = stack[-1]
                    del stack[-1]
                    #print('toanhang: ',toanhang, ' toantu:', character)
                    re = tinh_toan_mot_toan_hang(dic_input, toanhang, character)
                    #print('ket qua:' , re)
                    result_item.append(re)
                    stack.append(re)
            else:
                stack.append(character)
        #print('KET QUA: ',result_item)
        result.append(result_item)
        result_item = []
        final_postfix = backup_final_postfix.copy()

    '''final_result = ''
    for j in range(len(result)):
        for k in range(len(result[j])):
            final_result += str(result[j][k])
            if(k == len(result[j])-1):
                final_result += ';'
            else:
                final_result += ','
    final_result = final_result[:-1]'''
    #print (final_result)
    #return final_result
    return result

In [69]:
# chuyển từ trung tố sang hậu tố
ex = InFix2PostFix(ex)
print(ex)

PQ&~PQ||


In [41]:
# HÀM NÀY TỔNG HỢP HEADER VÀ BODY ĐỂ TẠO THÀNH BẢNG CHÂN TRỊ
def createTable(header, body):
    dic = {}
    for i in range(len(body[0])):
        list = []
        for j in range(len(body)):
            list.append(body[j][i])
        dic[header[i]] = list
    return dic

CÁC BIỂU THỨC LOGIC CHUẨN BỊ SẲN

    1. ~(P&Q)|(P|Q)
    2. (~P|(Q&R))>R
    3. ((P|~Q)>~P)=((P|~Q)>~P)

In [117]:
ex = '~(P&Q)|(P|Q)'

In [118]:
# convert from infix to postfix
ex = InFix2PostFix(ex)
print(ex)

PQ&~PQ||


In [119]:
# create header for truth table
header = createHeader(ex)
print(header)

['P', 'Q', 'P&Q', '~(P&Q)', 'P|Q', '~(P&Q)|(P|Q)']


In [120]:
# create body for truth table
body = createBody(ex)
print(body)

[[False, False, False, True, False, True], [False, True, False, True, True, True], [True, False, False, True, True, True], [True, True, True, False, True, True]]


In [121]:
# create truth table from header and body
table = createTable(header, body)

In [122]:
# print result to screen
import pandas as pd
df = pd.DataFrame(table)
df

,P,Q,P&Q,~(P&Q),P|Q,~(P&Q)|(P|Q)
0,False,False,False,True,False,True
1,False,True,False,True,True,True
2,True,False,False,True,True,True
3,True,True,True,False,True,True


In [ ]:
Code by Nguyễn Trọng Nhật